# Importamos las librerias

In [8]:
from _utils import *

# Usamos la `API` de **Messari** para obtener data en intervalos de tiempo de los halvings

Obtenemos las monedas mediante un endpoint del plan gratuito

In [9]:
coins_list = fetch_data(api=APIS.MESSARI,uri="marketdata/v1/assets/ath", parameters={})

In [10]:
coins_list = coins_list["data"]

In [11]:
len(coins_list)

15449

Tambien cargamos el dataset de `coins_info` para utilizar la informacion del `market_cap`, esta info ya esta limpia por lo que solo se encuentran monedas con `market_cap` mayores a `0.0` y no nulos 

In [13]:
coins_info = read_json("../../datasets/json/coins.json")

In [14]:
len(coins_info)

7862

Ahora filtramos los `coins_info` en base al `market_cap`

In [15]:
MARKET_CAP = 7_000_000 # Limite de market_cap en dolares

# Filtramos las monedas que no son mayores a X de market cap
low_cap_coins = list(
    filter(
        lambda x: x["market_cap"] <= MARKET_CAP, coins_info
    )
)
# Filtramos las monedas que son mayores a X de market cap
high_cap_coins = list(
    filter(
        lambda x: x["market_cap"] >= MARKET_CAP, coins_info
    )
)
# Observamos cuantas monedas hay en cada grupo
len(low_cap_coins), len(high_cap_coins)

(4117, 3745)

Observamos el `max_cap` de `low_cap_coins` y lo comparamos con el `max_cap` de `high_cap_coins`

In [16]:
max([x["market_cap"] for x in low_cap_coins]),max([x["market_cap"] for x in high_cap_coins])

(6999005, 1368315899533)

Ahora obtenemos los names en base a `low_cap_coins` y `high_cap_coins` para filtrar `coins_list`

In [17]:
# Creamos el set para los `names` de low_cap
low_coins_info_names = set()
list(map(lambda x: low_coins_info_names.add(x["name"]), low_cap_coins))

# Creamos el set para los `names` de high_cap
high_coins_info_names = set()
list(map(lambda x: high_coins_info_names.add(x["name"]), high_cap_coins))

# Obtenemos el tamaño de cada set
len(low_coins_info_names), len(high_coins_info_names)

(1847, 1547)

Ahora que tenemos `low_coins_info_names` y `high_coins_info_names` podemos filtrar `coins_list` para obtener `low_coins_list` y `high_coins_list`

In [18]:
low_coins = list(filter(lambda x: x["name"] in low_coins_info_names, coins_list))
len(low_coins)

1099

In [19]:
high_coins = list(filter(lambda x: x["name"] in high_coins_info_names, coins_list))
len(high_coins)

1030

Usamos el endpoint `marketdata/v1/assets/{assetId}/price/time-series` para obtener la data

In [21]:
def get_marketdata_per_halving(days:int, interval:str, coins: list)->dict[str,list]:
    halving_data: dict[str,list] = {}
    for halving in HALVINGS_DATE:
        print(f"halving {halving.value} ")
        halving_data[halving.value] = [] 
        start_timestamp, end_timestamp = get_start_end_timestamps(halving.value, days)
        print(f"star: {start_timestamp}, end: {end_timestamp}")
        for coin in coins:
            endpoint = f"marketdata/v1/assets/{coin["id"]}/price/time-series?interval={interval}&startTime={start_timestamp}&endTime={end_timestamp}"
            response = fetch_data(APIS.MESSARI, endpoint, parameters={})
            coin["halving"] = response["data"]
            halving_data[halving.value].append(coin)
            time.sleep(0.8)
    return halving_data

In [23]:
halving_data: dict[str,list] = get_marketdata_per_halving(DAYS_INTERVAL, "1d", low_coins)
save_json(halving_data, "./metadata/low_halving.json")
halving_data = get_marketdata_per_halving(DAYS_INTERVAL, "1d", high_coins)
save_json(halving_data, "./metadata/high_halving.json")

halving 2/12/2012 
star: 1354424400, end: 1376024400
halving 2/07/2016 
star: 1467435600, end: 1489035600
halving 3/05/2020 
star: 1588482000, end: 1610082000
halving 20/04/2024 
star: 1713589200, end: 1735189200
halving 2/12/2012 
star: 1354424400, end: 1376024400
halving 2/07/2016 
star: 1467435600, end: 1489035600
halving 3/05/2020 
star: 1588482000, end: 1610082000
halving 20/04/2024 
star: 1713589200, end: 1735189200
